# 🧠 MINERVA Model Training

**Strategic Pattern Analysis Specialist**

This notebook trains the MINERVA model, which specializes in:
- Pattern recognition and analysis
- Strategic reasoning
- Global pattern understanding
- Grid attention mechanisms
- Object relationship analysis

## 🚀 Setup Environment

In [ ]:
# Clone repository and install dependencies
!git clone https://github.com/AutomataControls/AutomataNexus_Olympus_AGI2.git /content/AutomataNexus_Olympus_AGI2
!cd /content/AutomataNexus_Olympus_AGI2 && pip install -r requirements.txt -q

In [ ]:
# Verify environment
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 🎯 Start MINERVA Training

Training configuration:
- **Architecture**: Grid Attention + Relational Networks
- **Parameters**: ~2.1M
- **Batch Size**: 512 (effective 2048 with gradient accumulation)
- **Epochs**: 300
- **Learning Rate**: 0.005 with ReduceLROnPlateau
- **✅ Fixed**: Stage 1+ DataLoader configuration for large datasets

In [ ]:
# Train MINERVA model
!cd /content/AutomataNexus_Olympus_AGI2 && python scripts/training/train_minerva.py

## 📊 Monitor Training Progress

In [ ]:
# Check GPU usage and training progress
import subprocess
import os
import glob
import time

# Show GPU status
gpu_info = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print("GPU Status:")
print(gpu_info.stdout)

# Check for MINERVA checkpoints
checkpoints = glob.glob('/content/AutomataNexus_Olympus_AGI2/arc_models_v4/minerva_*.pt')
if checkpoints:
    print("\nMINERVA Checkpoints:")
    for checkpoint in sorted(checkpoints):
        size = os.path.getsize(checkpoint) / (1024**2)  # Size in MB
        mtime = os.path.getmtime(checkpoint)
        print(f"  {os.path.basename(checkpoint)}: {size:.1f} MB - {time.ctime(mtime)}")

In [ ]:
# 🎯 Iterative Training with Custom Parameters
import subprocess
import tempfile
import os

# Modify these parameters as needed:
LEARNING_RATE = suggested_params.get('learning_rate', 0.005)
BATCH_SIZE = suggested_params.get('batch_size', 512)
EPOCHS_PER_STAGE = suggested_params.get('epochs_per_stage', 100)
TRANSFORMATION_PENALTY = suggested_params.get('transformation_penalty', 0.5)  # REDUCED from 1.0
EXACT_MATCH_BONUS = suggested_params.get('exact_match_bonus', 10.0)  # INCREASED from 5.0
GRADIENT_ACCUMULATION_STEPS = suggested_params.get('gradient_accumulation_steps', 4)
USE_MEPT = suggested_params.get('use_mept', True)
USE_LEAP = suggested_params.get('use_leap', True)
USE_PRISM = suggested_params.get('use_prism', True)
TARGET_ACCURACY = suggested_params.get('target_accuracy', 99.0)  # NEW: 99% target!

print(f"🚀 Starting MINERVA training iteration {len(trainer.iterations) + 1}")
print(f"Parameters: LR={LEARNING_RATE}, BS={BATCH_SIZE}, Epochs/Stage={EPOCHS_PER_STAGE}")
print(f"Loss weights: Transform penalty={TRANSFORMATION_PENALTY}, Exact bonus={EXACT_MATCH_BONUS}")
print(f"Features: MEPT={USE_MEPT}, LEAP={USE_LEAP}, PRISM={USE_PRISM}")
print(f"🎯 Target accuracy: {TARGET_ACCURACY}%")

# Create modified training script with custom parameters
modified_script = f"""
import sys
sys.path.append('/content/AutomataNexus_Olympus_AGI2')
sys.path.append('/content/AutomataNexus_Olympus_AGI2/src')

# Override parameters
import scripts.training.train_minerva as train_module
train_module.LEARNING_RATE = {LEARNING_RATE}
train_module.BATCH_SIZE = {BATCH_SIZE}
train_module.EPOCHS_PER_STAGE = {EPOCHS_PER_STAGE}
train_module.TRANSFORMATION_PENALTY = {TRANSFORMATION_PENALTY}
train_module.EXACT_MATCH_BONUS = {EXACT_MATCH_BONUS}
train_module.GRADIENT_ACCUMULATION_STEPS = {GRADIENT_ACCUMULATION_STEPS}
train_module.USE_MEPT = {USE_MEPT}
train_module.USE_LEAP = {USE_LEAP}
train_module.USE_PRISM = {USE_PRISM}

# Override inject_exact_match_training to use 99% target
import scripts.training.stage0_exact_match_boost as boost_module
original_inject = boost_module.inject_exact_match_training
def inject_with_target(*args, **kwargs):
    kwargs['target_accuracy'] = {TARGET_ACCURACY}
    return original_inject(*args, **kwargs)
boost_module.inject_exact_match_training = inject_with_target

# Run training
train_module.train_minerva()
"""

# Write and execute the modified script
with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
    f.write(modified_script)
    script_path = f.name

try:
    # Run the training with extended timeout for V4 mega-scale (12 hours max)
    print("⏰ Training timeout: 12 hours (V4 mega-scale)")
    result = subprocess.run(
        ['python', script_path],
        cwd='/content/AutomataNexus_Olympus_AGI2',
        capture_output=True,
        text=True,
        timeout=43200  # 12 hours for V4 mega-scale training
    )
    
    print("Training output:")
    print(result.stdout)
    if result.stderr:
        print("Errors:")
        print(result.stderr)
        
finally:
    # Clean up
    if os.path.exists(script_path):
        os.unlink(script_path)

In [ ]:
# 📊 Log Results and Update History
import glob
import torch
import json

# Collect results from the latest training
current_params = {
    'learning_rate': LEARNING_RATE,
    'batch_size': BATCH_SIZE,
    'epochs_per_stage': EPOCHS_PER_STAGE,
    'transformation_penalty': TRANSFORMATION_PENALTY,
    'exact_match_bonus': EXACT_MATCH_BONUS,
    'gradient_accumulation_steps': GRADIENT_ACCUMULATION_STEPS,
    'use_mept': USE_MEPT,
    'use_leap': USE_LEAP,
    'use_prism': USE_PRISM
}

# Load latest checkpoint to get results
model_files = glob.glob('/content/AutomataNexus_Olympus_AGI2/arc_models_v4/minerva_best.pt')
results = {}

if model_files:
    checkpoint = torch.load(model_files[0], map_location='cpu')
    results = {
        'best_exact': checkpoint.get('val_exact', 0),
        'best_val_loss': checkpoint.get('val_loss', float('inf')),
        'training_epoch': checkpoint.get('epoch', 0),
        'stage': checkpoint.get('stage', 0)
    }
    
    print(f"✅ Training completed!")
    print(f"   Best exact match: {results['best_exact']:.2f}%")
    print(f"   Best validation loss: {results['best_val_loss']:.4f}")
    print(f"   Training epochs: {results['training_epoch']}")
    
    # Log this iteration
    trainer.log_iteration(current_params, results)
    
    # Display updated history
    print("\n" + "="*60)
    trainer.display_history()
    
    # Provide suggestions for next iteration
    print("\n💡 Suggestions for next iteration:")
    next_params = trainer.suggest_next_params()
    
    current_exact = results['best_exact']
    best_iteration = trainer.get_best_iteration()
    best_exact = best_iteration['results']['best_exact'] if best_iteration else 0
    
    if current_exact >= best_exact * 0.95:  # Within 5% of best
        print("   🎯 Performance is good! Try:")
        if current_exact > best_exact:
            print(f"   ✅ New best! Consider fine-tuning with LR={next_params['learning_rate']:.4f}")
        else:
            print(f"   🔄 Try slightly different parameters or increase training epochs")
    else:
        print("   ⚠️ Performance below expectations. Suggestions:")
        print(f"   📉 Reduce learning rate to {next_params['learning_rate']:.4f}")
        print(f"   🎯 Increase exact match bonus to {next_params['exact_match_bonus']:.1f}")
        print("   💭 Consider enabling all features (MEPT, LEAP, PRISM)")
    
else:
    print("❌ No model checkpoint found. Training may have failed.")
    
print("\n🔄 To continue training, modify parameters above and re-run the training cell.")

In [ ]:
# 🎯 Iterative Training with Custom Parameters
import subprocess
import tempfile
import os

# Modify these parameters as needed:
LEARNING_RATE = suggested_params.get('learning_rate', 0.005)
BATCH_SIZE = suggested_params.get('batch_size', 512)
EPOCHS_PER_STAGE = suggested_params.get('epochs_per_stage', 100)
TRANSFORMATION_PENALTY = suggested_params.get('transformation_penalty', 0.5)  # REDUCED from 1.0
EXACT_MATCH_BONUS = suggested_params.get('exact_match_bonus', 10.0)  # INCREASED from 5.0
GRADIENT_ACCUMULATION_STEPS = suggested_params.get('gradient_accumulation_steps', 4)
USE_MEPT = suggested_params.get('use_mept', True)
USE_LEAP = suggested_params.get('use_leap', True)
USE_PRISM = suggested_params.get('use_prism', True)
TARGET_ACCURACY = suggested_params.get('target_accuracy', 99.0)  # NEW: 99% target!

print(f"🚀 Starting MINERVA training iteration {len(trainer.iterations) + 1}")
print(f"Parameters: LR={LEARNING_RATE}, BS={BATCH_SIZE}, Epochs/Stage={EPOCHS_PER_STAGE}")
print(f"Loss weights: Transform penalty={TRANSFORMATION_PENALTY}, Exact bonus={EXACT_MATCH_BONUS}")
print(f"Features: MEPT={USE_MEPT}, LEAP={USE_LEAP}, PRISM={USE_PRISM}")
print(f"🎯 Target accuracy: {TARGET_ACCURACY}%")

# Create modified training script with custom parameters
modified_script = f"""
import sys
sys.path.append('/content/AutomataNexus_Olympus_AGI2')
sys.path.append('/content/AutomataNexus_Olympus_AGI2/src')

# Override parameters
import scripts.training.train_minerva as train_module
train_module.LEARNING_RATE = {LEARNING_RATE}
train_module.BATCH_SIZE = {BATCH_SIZE}
train_module.EPOCHS_PER_STAGE = {EPOCHS_PER_STAGE}
train_module.TRANSFORMATION_PENALTY = {TRANSFORMATION_PENALTY}
train_module.EXACT_MATCH_BONUS = {EXACT_MATCH_BONUS}
train_module.GRADIENT_ACCUMULATION_STEPS = {GRADIENT_ACCUMULATION_STEPS}
train_module.USE_MEPT = {USE_MEPT}
train_module.USE_LEAP = {USE_LEAP}
train_module.USE_PRISM = {USE_PRISM}

# Override inject_exact_match_training to use 99% target
import scripts.training.stage0_exact_match_boost as boost_module
original_inject = boost_module.inject_exact_match_training
def inject_with_target(*args, **kwargs):
    kwargs['target_accuracy'] = {TARGET_ACCURACY}
    return original_inject(*args, **kwargs)
boost_module.inject_exact_match_training = inject_with_target

# Run training
train_module.train_minerva()
"""

# Write and execute the modified script
with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
    f.write(modified_script)
    script_path = f.name

try:
    # Run the training with extended timeout for V4 mega-scale (12 hours max)
    print("⏰ Training timeout: 12 hours (V4 mega-scale)")
    result = subprocess.run(
        ['python', script_path],
        cwd='/content/AutomataNexus_Olympus_AGI2',
        capture_output=True,
        text=True,
        timeout=43200  # 12 hours for V4 mega-scale training - KEEPING YOUR ORIGINAL!
    )
    
    print("Training output:")
    print(result.stdout)
    if result.stderr:
        print("Errors:")
        print(result.stderr)
        
finally:
    # Clean up
    if os.path.exists(script_path):
        os.unlink(script_path)

In [ ]:
# 🎛️ Hyperparameter Configuration
print("🎛️ Current Hyperparameter Configuration:")
print("=" * 50)

# Get suggestions for next iteration
suggested_params = trainer.suggest_next_params()

# Display current vs suggested parameters
for param, value in suggested_params.items():
    print(f"{param:25}: {value}")

print("\n💡 Automated Suggestions:")
if len(trainer.iterations) >= 1:
    latest = trainer.iterations[-1]
    best = trainer.get_best_iteration()
    
    if best and latest != best:
        print(f"   Latest performance: {latest['results'].get('best_exact', 0):.2f}%")
        print(f"   Best performance: {best['results'].get('best_exact', 0):.2f}%")
        
        if latest['results'].get('best_exact', 0) < best['results'].get('best_exact', 0):
            print("   🔄 Consider reverting to best parameters and reducing learning rate")
        else:
            print("   ✅ Performance is improving - continue with current approach")
    else:
        print("   🚀 Starting fresh - using default parameters")
else:
    print("   🆕 First iteration - using default parameters")

print("\n" + "=" * 50)
print("Modify parameters above if needed, then run the training cell below")

In [ ]:
# 🔧 Iterative Training Controller
import json
import os
import datetime
from typing import Dict, List

class MinervaIterativeTrainer:
    def __init__(self, base_dir='/content/AutomataNexus_Olympus_AGI2'):
        self.base_dir = base_dir
        self.iteration_log_file = f"{base_dir}/arc_models_v4/minerva_iterations.json"
        self.current_params = self.load_default_params()
        self.iterations = self.load_iteration_history()
    
    def load_default_params(self) -> Dict:
        return {
            'learning_rate': 0.005,
            'batch_size': 512,
            'epochs_per_stage': 100,
            'transformation_penalty': 1.0,
            'exact_match_bonus': 5.0,
            'gradient_accumulation_steps': 4,
            'use_mept': True,
            'use_leap': True,
            'use_prism': True
        }
    
    def load_iteration_history(self) -> List[Dict]:
        if os.path.exists(self.iteration_log_file):
            with open(self.iteration_log_file, 'r') as f:
                return json.load(f)
        return []
    
    def save_iteration_history(self):
        os.makedirs(os.path.dirname(self.iteration_log_file), exist_ok=True)
        with open(self.iteration_log_file, 'w') as f:
            json.dump(self.iterations, f, indent=2)
    
    def log_iteration(self, params: Dict, results: Dict):
        iteration = {
            'iteration': len(self.iterations) + 1,
            'timestamp': datetime.datetime.now().isoformat(),
            'parameters': params.copy(),
            'results': results.copy()
        }
        self.iterations.append(iteration)
        self.save_iteration_history()
    
    def get_best_iteration(self) -> Dict:
        if not self.iterations:
            return None
        return max(self.iterations, key=lambda x: x['results'].get('best_exact', 0))
    
    def suggest_next_params(self) -> Dict:
        if len(self.iterations) < 2:
            return self.current_params
        
        best = self.get_best_iteration()
        latest = self.iterations[-1]
        
        suggestions = best['parameters'].copy()
        
        # Adaptive suggestions based on performance
        latest_exact = latest['results'].get('best_exact', 0)
        best_exact = best['results'].get('best_exact', 0)
        
        if latest_exact < best_exact * 0.8:  # Performance dropped significantly
            # Reduce learning rate
            suggestions['learning_rate'] *= 0.7
            # Increase exact match bonus
            suggestions['exact_match_bonus'] = min(8.0, suggestions['exact_match_bonus'] * 1.2)
        elif latest_exact > best_exact * 1.1:  # Good improvement
            # Slightly increase learning rate
            suggestions['learning_rate'] = min(0.01, suggestions['learning_rate'] * 1.1)
        
        return suggestions
    
    def display_history(self):
        if not self.iterations:
            print("No iterations found.")
            return
        
        print("📈 MINERVA Training History:")
        print("-" * 80)
        for i, iteration in enumerate(self.iterations):
            exact = iteration['results'].get('best_exact', 0)
            loss = iteration['results'].get('best_val_loss', float('inf'))
            lr = iteration['parameters'].get('learning_rate', 0)
            timestamp = iteration['timestamp'][:16]  # Just date and hour
            
            status = "🟢 BEST" if iteration == self.get_best_iteration() else "⚪"
            print(f"{status} Iteration {i+1}: {exact:.2f}% exact | Loss: {loss:.4f} | LR: {lr:.4f} | {timestamp}")
        
        print("-" * 80)
        best = self.get_best_iteration()
        if best:
            print(f"🏆 Best: Iteration {best['iteration']} with {best['results']['best_exact']:.2f}% exact match")

# Initialize trainer
trainer = MinervaIterativeTrainer()
trainer.display_history()

## 🔄 Iterative Training & Hyperparameter Tuning

**Enhanced Training Loop with Checkpoint Resuming**

This section allows you to:
- Resume training from checkpoints
- Adjust hyperparameters between iterations
- Track performance improvements
- Get automated suggestions for next iteration

In [ ]:
# Load and evaluate MINERVA model
import torch
import json
import glob

# Find MINERVA model file
model_files = glob.glob('/content/AutomataNexus_Olympus_AGI2/arc_models_v4/minerva_best.pt')

if model_files:
    model_file = model_files[0]
    checkpoint = torch.load(model_file, map_location='cpu')
    
    print("🧠 MINERVA Training Results:")
    print(f"  Best Validation Exact Match: {checkpoint.get('val_exact', 'N/A'):.2f}%")
    print(f"  Training Epoch: {checkpoint.get('epoch', 'N/A')}")
    print(f"  Stage: {checkpoint.get('stage', 'N/A')}")
    print(f"  Model Size: {os.path.getsize(model_file) / (1024**2):.1f} MB")
else:
    print("❌ MINERVA model not found. Training may still be in progress.")

# Check training reports
report_files = glob.glob('/content/AutomataNexus_Olympus_AGI2/arc_models_v4/minerva_training_report_*.json')
if report_files:
    latest_report = sorted(report_files)[-1]
    with open(latest_report, 'r') as f:
        report = json.load(f)
    print(f"\n📊 Latest Training Report:")
    print(f"  Best Exact Match: {report.get('best_exact', 'N/A'):.2f}%")
    print(f"  Final Validation Loss: {report.get('best_val_loss', 'N/A'):.4f}")
    print(f"  Training Duration: {report.get('training_time', 'N/A')}")

## 🔧 Troubleshooting

If you encounter issues:

In [ ]:
# Clear GPU memory if needed
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

print(f"Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
print(f"Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved()) / 1e9:.2f} GB")

---

## ✅ Training Complete!

Your MINERVA model has been trained and saved to `/content/AutomataNexus_Olympus_AGI2/arc_models_v4/`

**Next Steps:**
- Train other OLYMPUS models (ATLAS, IRIS, CHRONOS, PROMETHEUS)
- Use the ensemble training notebook for combined training
- Test the model on ARC evaluation tasks

**AutomataNexus OLYMPUS AGI2** - *Where Neural Networks Meet Symbolic Logic*